In [ ]:
import os
os.chdir('..')

import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor

class BhajanSpider(CrawlSpider):
    
    name = "Bhajans"
    # allowed_domains = ['sairhythms.org/song']
    # base_url = ['"https:/www.sairhythms.org/']

    start_urls = [
        'https://sairhythms.org/'

    ]
    
    rules = (
        Rule(LinkExtractor(allow = "/song/"), callback='parse_item', follow=True),
    )
    
    def parse_item(self, response):
        def extract_with_xpath(query):
            return response.xpath(query).xpath('string(.)').get()
        
        return {
            'Link': response.xpath("""//div[contains(@class, 'audio-link')]//a[1]""").xpath('text()').get(),
            'Deity': extract_with_xpath("""//div[contains(@class, 'deity-row')]//div[2]"""),
            'Language': extract_with_xpath("""//div[contains(@class, 'language-row')]//div[2]"""),
            'Raga': extract_with_xpath("""//div[contains(@class, 'raga-title')]"""), # excludes other ragas for songs with multiple ragas (only includes first)
            # 'Raga': extract_with_xpath("""//div[contains(@class, 'raga-row')]//div[2]"""), # This gets all text but includes noise
            'Beat': extract_with_xpath("""//div[contains(@class, 'beat-row')]//div[2]"""),
            'Level': extract_with_xpath("""//div[contains(@class, 'level-row')]//div[2]"""),
            'Tempo': extract_with_xpath("""//div[contains(@class, 'tempo-row')]//div[2]""")
        }

In [ ]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'FEED_FORMAT': 'json',
    'FEED_URI': 'data/raw/bhajans_info.json'
})

process.crawl(BhajanSpider)
process.start()